# Option Pricing with with PyTorch on GPU

Copyright Matthias Groncki, 2018

This is a port of one of my previous blog posts about using TensorFlow to price options.

After using PyTorch for another project, i was impressed how straight forward it is, so I've decided to revisit my previous examples and use PyTorch this time

In [78]:
import numpy as np
import torch
import datetime as dt

## Monte Carlo Pricing for Single Barrier Option on a GPU vs CPU

In [12]:
def monte_carlo_down_out_py(S_0, strike, time_to_expiry, implied_vol, riskfree_rate, barrier, steps, samples):
    stdnorm_random_variates = np.random.randn(samples, steps)
    S = S_0
    K = strike
    dt = time_to_expiry / stdnorm_random_variates.shape[1]
    sigma = implied_vol
    r = riskfree_rate
    B = barrier
    # See Advanced Monte Carlo methods for barrier and related exotic options by Emmanuel Gobet
    B_shift = B*np.exp(0.5826*sigma*np.sqrt(dt))
    S_T = S * np.cumprod(np.exp((r-sigma**2/2)*dt+sigma*np.sqrt(dt)*stdnorm_random_variates), axis=1)
    non_touch = (np.min(S_T, axis=1) > B_shift)*1
    call_payout = np.maximum(S_T[:,-1] - K, 0)
    npv = np.mean(non_touch * call_payout)
    return np.exp(-time_to_expiry*r)*npv

In [13]:
%%timeit
monte_carlo_down_out_py(100., 110., 2., 0.2, 0.03, 90., 1000, 100000)

8.24 s ± 22.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [83]:
def monte_carlo_down_out_torch_cuda(S_0, strike, time_to_expiry, implied_vol, riskfree_rate, barrier, steps, samples):
    stdnorm_random_variates = torch.cuda.FloatTensor(steps, samples).normal_()
    S = S_0
    K = strike
    dt = time_to_expiry / stdnorm_random_variates.shape[1]
    sigma = implied_vol
    r = riskfree_rate
    B = barrier
    # See Advanced Monte Carlo methods for barrier and related exotic options by Emmanuel Gobet
    B_shift = B*torch.exp(0.5826*sigma*torch.sqrt(dt))
    S_T = S * torch.cumprod(torch.exp((r-sigma**2/2)*dt+sigma*torch.sqrt(dt)*stdnorm_random_variates), dim=1)
    non_touch = torch.min(S_T, dim=1)[0] > B_shift
    non_touch = non_touch.type(torch.cuda.FloatTensor)
    call_payout = S_T[:,-1] - K
    call_payout[call_payout<0]=0
    npv = torch.mean(non_touch * call_payout)
    return torch.exp(-time_to_expiry*r)*npv

In [84]:
%%timeit
S = torch.tensor([100.],requires_grad=True, device='cuda')
K = torch.tensor([110.],requires_grad=True, device='cuda')
T = torch.tensor([2.],requires_grad=True, device='cuda')
sigma = torch.tensor([0.2],requires_grad=True, device='cuda')
r = torch.tensor([0.03],requires_grad=True, device='cuda')
B = torch.tensor([90.],requires_grad=True, device='cuda')
monte_carlo_down_out_torch_cuda(S, K, T, sigma, r, B, 1000, 100000)

56.3 ms ± 89.1 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [115]:
%%timeit
S = torch.tensor([100.],requires_grad=True, device='cuda')
K = torch.tensor([110.],requires_grad=True, device='cuda')
T = torch.tensor([2.],requires_grad=True, device='cuda')
sigma = torch.tensor([0.2],requires_grad=True, device='cuda')
r = torch.tensor([0.03],requires_grad=True, device='cuda')
B = torch.tensor([90.],requires_grad=True, device='cuda')
npv_torch_mc = monte_carlo_down_out_torch_cuda(S, K, T, sigma, r, B, 1000, 100000)
npv_torch_mc.backward()

160 ms ± 94 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
